CS4001/4042 Assignment 1
---
Part A, Q1 (15 marks)
---

>Design a feedforward deep neural network (DNN) which consists of **three** hidden layers of 128 neurons each with ReLU activation function, and an output layer with sigmoid activation function. Apply dropout of probability **0.2** to each of the hidden layers. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import time
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from scipy.io import wavfile as wav

from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

from common_utils import set_seed

# setting seed
set_seed(42)

Define the model class.

In [ ]:
# TODO: Enter your code here
class MLP(nn.Module):
    def __init__(self, input_size, first_hidden_size=None, hidden_size=128, output_size=1, dropout_prob=0.2):
        super(MLP, self).__init__()
        if first_hidden_size == None:
            self.fc1 = nn.Linear(input_size, hidden_size)
            self.fc2 = nn.Linear(hidden_size, hidden_size)
        else:
            self.fc1 = nn.Linear(input_size, first_hidden_size)
            self.fc2 = nn.Linear(first_hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.relu(self.fc3(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc4(x))
        return x

> Divide the dataset into a 70:30 ratio for training and testing. Use **appropriate** scaling of input features. We solely assume that there are only two datasets here: training & test.

Split the dataset and do preprocessing. You can use the split_dataset and preprocess_dataset provided for you.

In [ ]:
from common_utils import split_dataset, preprocess_dataset

# TODO: Enter your code here
def load_csv(file_path: str) -> pd.DataFrame:
    df = pd.read_csv(file_path)
    filename_col = df["filename"]
    lable_col = filename_col.apply(lambda x: x.split(".")[0])
    df["label"] = lable_col
    df.drop(["filename"], axis=1, inplace=True)
    return df

def load_dataframe_split(df: pd.DataFrame) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    df_train, y_train, df_test, y_test = split_dataset(df, ["label"], 0.3, 42)
    df_train, df_test = preprocess_dataset(df_train, df_test)
    return df_train, y_train, df_test, y_test

df = load_csv("./audio_gtzan.csv")

X_train, y_train, X_test, y_test = load_dataframe_split(df)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

> Use the training dataset to train the model for 100 epochs. Use a mini-batch gradient descent with **‘Adam’** optimizer with learning rate of **0.001**, and **batch size = 128**. Implement early stopping with patience of **3**.

1. Define a Pytorch Dataset and Dataloaders.  

In [ ]:
# TODO: Enter your code here
class AudioDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        if self.X.shape[0] != self.y.shape[0]:
            raise ValueError("X and y must have the same length.")
        if self.y.ndim == 1:
            self.y = self.y.reshape(-1, 1)
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
class AudioDataLoader(DataLoader):
    def __init__(self, dataset, batch_size, shuffle=True):
        super(AudioDataLoader, self).__init__(dataset, batch_size, shuffle=shuffle)
        self.shuffle = shuffle

    def __iter__(self):
        for i in range(0, len(self.dataset), self.batch_size):
            yield self.dataset[i : i + self.batch_size]
            
    def __str__(self):
        dataset_info = f"\tDataset length: {len(self.dataset)}, \n\tDataset shape: {self.dataset[0][0].shape}, \n\tBatch size: {self.batch_size}, \n\tShuffle: {self.shuffle}"
        return f"AudioDataLoader(\n{dataset_info}\n)\n"
            
train_dataset = AudioDataset(X_train, y_train)
test_dataset = AudioDataset(X_test, y_test)

train_loader = AudioDataLoader(train_dataset, 128)
test_loader = AudioDataLoader(test_dataset, 128)

print(train_loader, test_loader)

2. Next, define the model, optimizer and loss function.

In [ ]:
# TODO: Enter your code here
model = MLP(57)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_fn = nn.BCELoss()

3. Train model for 100 epochs. Record down train and test accuracies. Implement early stopping.

In [ ]:
from common_utils import EarlyStopper

# TODO: Enter your code here
early_stopper = EarlyStopper(patience=3, min_delta=1e-5)
n_epochs = 100
acc_train = []
acc_test = []
loss_train = []
loss_test = []
f1_train = []
f1_test = []
precision_train = []
precision_test = []
recall_train = []
recall_test = []


for epoch in range(n_epochs):
    model.train()
    with torch.enable_grad():
        y_pred_list = []
        y_true_list = []
        for batch in train_loader:
            X, y = batch
            y_pred = model(X)
            loss = loss_fn(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            y_pred_list.append(y_pred)
            y_true_list.append(y)
        y_pred_list = torch.cat(y_pred_list)
        y_true_list= torch.cat(y_true_list)
        loss_train.append(loss_fn(y_pred_list, y_true_list))
        y_pred_list = torch.round(y_pred_list)
        acc_train.append((y_pred_list == y_true_list).sum().item() / len(y_true_list))
        y_pred_list, y_true_list = y_pred_list.detach().numpy(), y_true_list.detach().numpy()
        f1_train.append(f1_score(y_true_list, y_pred_list))
        precision_train.append(precision_score(y_true_list, y_pred_list))
        recall_train.append(recall_score(y_true_list, y_pred_list))

    model.eval()
    with torch.no_grad():
        y_pred_list = []
        y_true_list = []
        for batch in test_loader:
            X, y = batch
            y_pred = model(X)
            loss = loss_fn(y_pred, y)
            y_pred_list.append(y_pred)
            y_true_list.append(y)
        y_pred_list = torch.cat(y_pred_list)
        y_true_list= torch.cat(y_true_list)
        loss_test.append(loss_fn(y_pred_list, y_true_list))
        y_pred_list = torch.round(y_pred_list)
        acc_test.append((y_pred_list == y_true_list).sum().item() / len(y_true_list))
        y_pred_list, y_true_list = y_pred_list.detach().numpy(), y_true_list.detach().numpy()
        f1_test.append(f1_score(y_true_list, y_pred_list))
        precision_test.append(precision_score(y_true_list, y_pred_list))
        recall_test.append(recall_score(y_true_list, y_pred_list))

    log_info = f"epoch: {epoch} \t| train loss: {loss_train[-1]:.5f} \t| test loss: {loss_test[-1]:.5f}"
    print(log_info)

    if early_stopper.early_stop(loss_test[-1]):
        print(f"early stopping at epoch {epoch}")
        break
    
loss_train = torch.tensor(loss_train)
loss_test = torch.tensor(loss_test)

> Plot train and test accuracies and losses on training and test data against training epochs and comment on the line plots.

In [ ]:
# TODO: Enter your code here

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, len(acc_train)+1), acc_train, label='train')
plt.plot(range(1, len(acc_test)+1), acc_test, label='test')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, len(loss_train)+1), loss_train, label='train')
plt.plot(range(1, len(loss_test)+1), loss_test, label='test')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

\# TODO: \<Enter your answer here\>

The train and test accuracy both increase with epochs, with train accuracy being higher than test accuracy. And the train and test loss both decrease with epochs, with train loss being lower than test loss. The model is doing well in fitting the training data, but the test accuracy is not as high as the train accuracy. Hence, the model may be overfitting the training data.

Part A, Q2 (10 marks)
---

In this question, we will determine the optimal batch size for mini-batch gradient descent. Find the optimal batch size for mini-batch gradient descent by training the neural network and evaluating the performances for different batch sizes. Note: Use 5-fold cross-validation on training partition to perform hyperparameter selection. You will have to reconsider the scaling of the dataset during the 5-fold cross validation.

To reduce repeated code, you may need to place the network (MLP defined in QA1) in a separate file called **common_utils.py**. Import it here for Q2. You will not be repenalised for any error in QA1 here as the code in QA1 will not be remarked. The following code cell will not be marked.

In [ ]:
# TODO: Enter your code here

batch_sizes = [32, 64, 128, 256]

model_dict = {batch_size: MLP(57) for batch_size in batch_sizes}

model_dict

> Plot mean cross-validation accuracies on the final epoch for different batch sizes as a scatter plot. Limit search space to batch sizes {32, 64, 128, 256}. Next, create a table of time taken to train the network on the last epoch against different batch sizes. Finally, select the optimal batch size and state a reason for your selection. This might take a while to run, so plan your time carefully.

1. Define different folds for different batch sizes to get a dictionary of training and validation datasets. Preprocess your datasets accordingly. Please use the following name conventions:
    - X_train_scaled_dict[batch_size] is a list of the preprocessed training matrix for the different folds. 
    - X_val_scaled_dict[batch_size] is a list of the processed validation matrix for the different folds. 
    - y_train_dict[batch_size] and y_val_dict[batch_size] is a list of labels for the different folds.

In [ ]:
# TODO: Enter your code here

X, y = X_train, y_train

X_train_scaled_dict = {batch_size: None for batch_size in batch_sizes}
X_val_scaled_dict = {batch_size: None for batch_size in batch_sizes}
y_train_dict = {batch_size: None for batch_size in batch_sizes}
y_val_dict = {batch_size: None for batch_size in batch_sizes}

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for batch_size in batch_sizes:
    X_train_folds = []
    X_val_folds = []
    y_train_folds = []
    y_val_folds = []

    for train_index, val_index in kf.split(X):
        X_train_fold, X_val_fold = X[train_index], X[val_index]
        y_train_fold, y_val_fold = y[train_index], y[val_index]

        scaler = preprocessing.StandardScaler()
        X_train_fold = scaler.fit_transform(X_train_fold)
        X_val_fold = scaler.transform(X_val_fold)

        X_train_folds.append(X_train_fold)
        X_val_folds.append(X_val_fold)
        y_train_folds.append(y_train_fold)
        y_val_folds.append(y_val_fold)

    X_train_scaled_dict[batch_size] = X_train_folds
    X_val_scaled_dict[batch_size] = X_val_folds
    y_train_dict[batch_size] = y_train_folds
    y_val_dict[batch_size] = y_val_folds
    
print(X_train_scaled_dict[32][0].shape, X_val_scaled_dict[32][0].shape, y_train_dict[32][0].shape, y_val_dict[32][0].shape)

2. Perform hyperparameter tuning for the different batch sizes with 5-fold cross validation. 

In [ ]:
# TODO: Enter your code here

time_train_dict = {batch_size: [] for batch_size in batch_sizes}
mean_cv_acc_dict = {batch_size: [] for batch_size in batch_sizes}

for batch_size in batch_sizes:
    set_seed(42)
    model = model_dict[batch_size]
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    loss_fn = nn.BCELoss()

    n_splits = 5
    n_epochs = 200       

    for i in range(n_splits):
        X_train_ndarray = X_train_scaled_dict[batch_size][i]
        X_val_ndarray = X_val_scaled_dict[batch_size][i]
        y_train_ndarray = y_train_dict[batch_size][i]
        y_val_ndarray = y_val_dict[batch_size][i]
        train_dataset = AudioDataset(X_train_ndarray, y_train_ndarray)
        val_dataset = AudioDataset(X_val_ndarray, y_val_ndarray)
        train_loader = AudioDataLoader(train_dataset, batch_size)
        val_loader = AudioDataLoader(val_dataset, batch_size)

        loss_train = []
        acc_train = []
        time_train = []
        loss_val = []
        acc_val = []

        early_stopper = EarlyStopper(patience=3, min_delta=1e-5)

        for epoch in range(n_epochs):
            start_time = time.time()
            model.train()
            with torch.enable_grad():
                y_pred_list = []
                y_true_list = []
                for batch in train_loader:
                    X, y = batch
                    y_pred = model(X)
                    loss = loss_fn(y_pred, y)
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    y_pred_list.append(y_pred)
                    y_true_list.append(y)
                y_pred_list = torch.cat(y_pred_list)
                y_true_list= torch.cat(y_true_list)
                loss_train.append(loss_fn(y_pred_list, y_true_list))
                y_pred_list = torch.round(y_pred_list)
                acc_train.append((y_pred_list == y_true_list).sum().item() / len(y_true_list))
            end_time = time.time()
            time_train.append(end_time - start_time)

            model.eval()
            with torch.no_grad():
                y_pred_list = []
                y_true_list = []
                for batch in val_loader:
                    X, y = batch
                    y_pred = model(X)
                    loss = loss_fn(y_pred, y)
                    y_pred_list.append(y_pred)
                    y_true_list.append(y)
                y_pred_list = torch.cat(y_pred_list)
                y_true_list= torch.cat(y_true_list)
                loss_val.append(loss_fn(y_pred_list, y_true_list))
                y_pred_list = torch.round(y_pred_list)
                acc_val.append((y_pred_list == y_true_list).sum().item() / len(y_true_list))

            log_info = f"batch_size: {batch_size} \t| epoch: {epoch} \t| train loss: {loss_train[-1]:.5f} \t| val loss: {loss_val[-1]:.5f}\t | time taken: {time_train[-1]}"
            print(log_info)

            if early_stopper.early_stop(loss_val[-1]):
                print(f"early stopping at epoch {epoch}")
                break

            if epoch == n_epochs - 1:
                print("reach max number of epoch")

        time_train_dict[batch_size].append(time_train[-1])
        mean_cv_acc_dict[batch_size].append(acc_val[-1])

3. Plot scatterplot of mean cross validation accuracies on the final epoch for the different batch sizes. Limit search space to batch sizes {32, 64, 128, 256}.

In [ ]:
# TODO: Enter your code here

mean_cv_accs_plot = [np.mean(mean_cv_acc_dict[batch_size]) for batch_size in batch_sizes]

plt.scatter(batch_sizes, mean_cv_accs_plot)
plt.xlabel("Batch Size")
plt.ylabel("Mean Cross Validation Accuracy")
plt.title("Mean Cross Validation Accuracy vs Batch Size")
plt.show()

4. Create a table of time taken to train the network on the last epoch against different batch sizes. 

In [ ]:
# TODO: Enter your code here

batch_sizes = list(time_train_dict.keys())
split_numbers = [f"Split {i+1}" for i in range(len(time_train_dict[batch_sizes[0]]))]
columns = split_numbers + ["Mean"]

results = pd.DataFrame(index=batch_sizes, columns=columns)

for batch_size in batch_sizes:
    times = time_train_dict[batch_size]
    mean_time = np.mean(times)

    for i, time_count in enumerate(times):
        results.at[batch_size, f"Split {i+1}"] = time_count

    results.at[batch_size, "Mean"] = mean_time
    
print(results)

5. Select the optimal batch size and state a reason for your selection.

\# TODO: \<Enter your answer here\>

From the experimental chart, we can see that no matter how the batch size is set, a high cross-validation accuracy can be achieved. It is also worth noting that the cross-validation accuracy tends to decrease as the batch size increases. From the perspective of time consumption, a larger batch size consumes less time. Empirically, doubling the batch size will reduce the time consumption to about three-quarters of the original. Therefore, this is a decision-making problem. If we want the model to train faster, we can choose a larger batch. If we want the model to have better accuracy, we should set a smaller batch. In the scenario of this project, the absolute value of the change in accuracy is not obvious, so we can accept the slight decrease in accuracy caused by a larger batch. Therefore, I will choose a batch size of 32, because it brings faster training time and acceptable accuracy at the same time.

Part A, Q3 (10 marks)
---
In this question, we will find the optimal number of hidden neurons for first hidden layer of the 4-layer network (3 hidden layers, output layer) designed in Q1 and Q2.

To reduce repeated code, you may need to import the network (MLP defined in QA1) from common_utils.py. You will not be repenalised for any error in QA1 here as the code in QA1 will not be remarked. The following code cell will not be marked.

In [ ]:
# TODO: Enter your code here

first_hidden_sizes = [64, 128, 256]

model_dict = {first_hidden_size: MLP(57, first_hidden_size=first_hidden_size) for first_hidden_size in first_hidden_sizes}

model_dict

> Plot the mean cross-validation accuracies on the final epoch for different numbers of hidden-layer neurons using a scatter plot. Limit the search space of the number of neurons to {64, 128, 256}. Continue using 5-fold cross validation on training dataset. 

1. Perform hyperparameter tuning for the different neurons with 5-fold cross validation.

In [ ]:
# TODO: Enter your code here

X, y = X_train, y_train


X_train_scaled_list = []
X_val_scaled_list = []
y_train_list = []
y_val_list = []

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, val_index in kf.split(X):
    X_train_fold, X_val_fold = X[train_index], X[val_index]
    y_train_fold, y_val_fold = y[train_index], y[val_index]

    scaler = preprocessing.StandardScaler()
    X_train_fold = scaler.fit_transform(X_train_fold)
    X_val_fold = scaler.transform(X_val_fold)

    X_train_scaled_list.append(X_train_fold)
    X_val_scaled_list.append(X_val_fold)
    y_train_list.append(y_train_fold)
    y_val_list.append(y_val_fold)
    
print(
    X_train_scaled_list[0].shape,
    X_val_scaled_list[0].shape,
    y_train_list[0].shape,
    y_val_list[0].shape,
)

In [ ]:
# CONTINUE


time_train_dict = {first_hidden_size: [] for first_hidden_size in first_hidden_sizes}
mean_cv_acc_dict = {first_hidden_size: [] for first_hidden_size in first_hidden_sizes}
mean_test_acc_dict = {first_hidden_size: [] for first_hidden_size in first_hidden_sizes}
acc_train_dict = {first_hidden_size: [] for first_hidden_size in first_hidden_sizes}
acc_val_dict = {first_hidden_size: [] for first_hidden_size in first_hidden_sizes}
acc_test_dict = {first_hidden_size: [] for first_hidden_size in first_hidden_sizes}

for first_hidden_size in first_hidden_sizes:
    set_seed(42)
    model = model_dict[first_hidden_size]
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    loss_fn = nn.BCELoss()

    n_splits = 5
    n_epochs = 200
    best_batch_size = 32

    for i in range(n_splits):
        X_train_ndarray = X_train_scaled_list[i]
        X_val_ndarray = X_val_scaled_list[i]
        y_train_ndarray = y_train_list[i]
        y_val_ndarray = y_val_list[i]
        train_dataset = AudioDataset(X_train_ndarray, y_train_ndarray)
        val_dataset = AudioDataset(X_val_ndarray, y_val_ndarray)
        train_loader = AudioDataLoader(train_dataset, best_batch_size)
        val_loader = AudioDataLoader(val_dataset, best_batch_size)

        loss_train = []
        acc_train = []
        time_train = []
        loss_val = []
        acc_val = []
        loss_test = []
        acc_test = []

        early_stopper = EarlyStopper(patience=3, min_delta=1e-5)

        for epoch in range(n_epochs):
            start_time = time.time()
            model.train()
            with torch.enable_grad():
                y_pred_list = []
                y_true_list = []
                for batch in train_loader:
                    X, y = batch
                    y_pred = model(X)
                    loss = loss_fn(y_pred, y)
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    y_pred_list.append(y_pred)
                    y_true_list.append(y)
                y_pred_list = torch.cat(y_pred_list)
                y_true_list= torch.cat(y_true_list)
                loss_train.append(loss_fn(y_pred_list, y_true_list))
                y_pred_list = torch.round(y_pred_list)
                acc_train.append((y_pred_list == y_true_list).sum().item() / len(y_true_list))
            end_time = time.time()
            time_train.append(end_time - start_time)

            model.eval()
            with torch.no_grad():
                y_pred_list = []
                y_true_list = []
                for batch in val_loader:
                    X, y = batch
                    y_pred = model(X)
                    loss = loss_fn(y_pred, y)
                    y_pred_list.append(y_pred)
                    y_true_list.append(y)
                y_pred_list = torch.cat(y_pred_list)
                y_true_list= torch.cat(y_true_list)
                loss_val.append(loss_fn(y_pred_list, y_true_list))
                y_pred_list = torch.round(y_pred_list)
                acc_val.append((y_pred_list == y_true_list).sum().item() / len(y_true_list))

            model.eval()
            with torch.no_grad():
                y_pred_list = []
                y_true_list = []
                for batch in test_loader:
                    X, y = batch
                    y_pred = model(X)
                    loss = loss_fn(y_pred, y)
                    y_pred_list.append(y_pred)
                    y_true_list.append(y)
                y_pred_list = torch.cat(y_pred_list)
                y_true_list= torch.cat(y_true_list)
                loss_test.append(loss_fn(y_pred_list, y_true_list))
                y_pred_list = torch.round(y_pred_list)
                acc_test.append((y_pred_list == y_true_list).sum().item() / len(y_true_list))

            log_info = f"first_hidden_size: {first_hidden_size} \t| epoch: {epoch} \t| train loss: {loss_train[-1]:.5f} \t| val loss: {loss_val[-1]:.5f}\t| test loss: {loss_test[-1]:.5f}\t| time taken: {time_train[-1]}"
            print(log_info)

            if early_stopper.early_stop(loss_val[-1]):
                print(f"early stopping at epoch {epoch}")
                break

            if epoch == n_epochs - 1:
                print("reach max number of epoch")

        time_train_dict[first_hidden_size].append(time_train[-1])
        mean_cv_acc_dict[first_hidden_size].append(acc_val[-1])
        mean_test_acc_dict[first_hidden_size].append(acc_test[-1])
        acc_train_dict[first_hidden_size].append(acc_train)
        acc_val_dict[first_hidden_size].append(acc_val)
        acc_test_dict[first_hidden_size].append(acc_test)

2. Plot the mean cross-validation accuracies on the final epoch for different numbers of hidden-layer neurons using a scatter plot. Limit the search space of the number of neurons to {64, 128, 256}.

In [ ]:
# TODO: Enter your code here

mean_cv_accs_plot = [
    np.mean(mean_cv_acc_dict[first_hidden_size])
    for first_hidden_size in first_hidden_sizes
]
plt.scatter(first_hidden_sizes, mean_cv_accs_plot)
plt.xlabel("Number of Hidden Layer Neurons")
plt.ylabel("Mean Cross Validation Accuracy")
plt.title("Mean Cross Validation Accuracy vs Number of Hidden Layer Neurons")
plt.show()

> Select the optimal number of neurons for the hidden layer. State the rationale for your selection.

In [ ]:
# TODO: Enter your code here

n_splits = 5
for i in range(n_splits):
    mean_cv_accs_plot = [
        mean_cv_acc_dict[first_hidden_size][i]
        for first_hidden_size in first_hidden_sizes
    ]
    plt.scatter(first_hidden_sizes, mean_cv_accs_plot, label=f"Fold {i+1}")
plt.xlabel("Number of Hidden Layer Neurons")
plt.ylabel("Mean Cross Validation Accuracy")
plt.legend()
plt.title("Fold Accuracy vs Number of Hidden Layer Neurons")
plt.show()

\# TODO: \<Enter your answer here\>

From the chart, we can see that when the number of neurons in the first hidden layer is set to 256, the model has better accuracy on the validation set than the other two settings. This may be because the characteristics of the original input data are more complex, so a more complex model has a better fitting effect. Under this parameter setting, there is no obvious overfitting.

> Plot the train and test accuracies against training epochs with the optimal number of neurons using a line plot.
Note: use this optimal number of neurons for the rest of the experiments.

In [ ]:
# TODO: Enter your code here

optimal_first_hidden_size = 256
acc_train_full = []
acc_val_full = []
acc_test_full = []
for i in range(n_splits):
    plt.plot(range(1, len(acc_train_dict[optimal_first_hidden_size][i])+1), acc_train_dict[optimal_first_hidden_size][i], label='train')
    plt.plot(range(1, len(acc_val_dict[optimal_first_hidden_size][i])+1), acc_val_dict[optimal_first_hidden_size][i], label='val')
    plt.plot(
        range(1, len(acc_test_dict[optimal_first_hidden_size][i]) + 1),
        acc_test_dict[optimal_first_hidden_size][i],
        label="test",
    )
    plt.plot()
    acc_train_full += acc_train_dict[optimal_first_hidden_size][i]
    acc_val_full += acc_val_dict[optimal_first_hidden_size][i]
    acc_test_full += acc_test_dict[optimal_first_hidden_size][i]
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title(f"Fold {i + 1} Accuracy vs Epochs with {optimal_first_hidden_size} Hidden Layer Neurons")
    plt.show()

plt.plot(
    range(1, len(acc_train_full) + 1),
    acc_train_full,
    label="train",
)
plt.plot(
    range(1, len(acc_val_full) + 1),
    acc_val_full,
    label="val",
)
plt.plot(
    range(1, len(acc_test_full) + 1),
    acc_test_full,
    label="test",
)
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.title(
    f"Full Process Accuracy vs Epochs with {optimal_first_hidden_size} Hidden Layer Neurons"
)
plt.show()

Part A, Q4 (10 marks)
---
In this section, we will understand the utility of such a neural network for a test audio. 

Do a model prediction on the sample test audio and obtain the predicted label using a threshold of 0.5. The model used is the optimized pretrained model using the selected optimal batch size and optimal number of neurons. 
Find the most important features on the model prediction for the test sample using SHAP. Plot the local feature importance with a force plot and explain your observations.  (Refer to the documentation and these three useful references:
https://christophm.github.io/interpretable-ml-book/shap.html#examples-5, 
https://towardsdatascience.com/deep-learning-model-interpretation-using-shap-a21786e91d16,  
https://medium.com/mlearning-ai/shap-force-plots-for-classification-d30be430e195) 

To reduce repeated code, you may need to import the network (MLP defined in QA1) from **common_utils.py**. You will not be repenalised for any error in QA1 here as the code in QA1 will not be remarked. The following code cell will not be marked.

In [ ]:
# TODO: Enter your code here

model_select = model_dict[256]



> Install and import shap

In [ ]:
# TODO: Enter your code here

!pip install shap

In [ ]:
import shap

> Preprocess 'audio_test.wav' using the function 'extract_features' in common_utils.py. Please make sure the features are stored in a pandas dataframe, using variable name 'df', and fill the size of 'df' in 'size_row' and 'size_column'.

In [ ]:
import IPython
IPython.display.Audio("./audio_test.wav")

In [ ]:
# TODO: Enter your code here
from common_utils import extract_features

df = extract_features("audio_test.wav")
size_row = 1
size_column = 58
df

> Do a model prediction on the sample test audio and obtain the predicted label using a threshold of 0.5. The model used is the optimized pretrained model using the selected optimal batch size and optimal number of neurons. 

 1.  Preprocess to obtain the test data, save the test data as numpy array, print the shape of the test data.

In [ ]:
# TODO: Enter your code here
X_train, _, _, _ = split_dataset(load_csv("./audio_gtzan.csv"), ["label"], 0.3, 42)
X_test = df.drop(["filename"], axis=1).to_numpy()
X_train, X_test = preprocess_dataset(X_train, X_test)

X_train, X_test

2. Do a model prediction on the sample test audio and obtain the predicted label using a threshold of 0.5. The model used is the optimized pretrained model using the selected optimal batch size and optimal number of neurons. Note: Please define the variable of your final predicted label as 'pred_label'.

In [ ]:
# TODO: Enter your code here
pred = model(torch.tensor(X_test).float()).detach().numpy()[0]
pred_label = (pred > 0.5).astype(int)
pred, pred_label

> Find the most important features on the model prediction for your test sample using SHAP. Create an instance of the DeepSHAP which is called DeepExplainer using traianing dataset: https://shap-lrjball.readthedocs.io/en/latest/generated/shap.DeepExplainer.html.

Plot the local feature importance with a force plot and explain your observations.  (Refer to the documentation and these three useful references:
https://christophm.github.io/interpretable-ml-book/shap.html#examples-5, 
https://towardsdatascience.com/deep-learning-model-interpretation-using-shap-a21786e91d16,  
https://medium.com/mlearning-ai/shap-force-plots-for-classification-d30be430e195) 

In [ ]:
'''
Fit the explainer on a subset of the data (you can try all but then gets slower)
Return approximate SHAP values for the model applied to the data given by X.
Plot the local feature importance with a force plot and explain your observations.
'''
# TODO: Enter your code here

X_train, _, X_test, _ = split_dataset(load_csv("./audio_gtzan.csv"), ["label"], 0.3, 42)
X_train, X_test = preprocess_dataset(X_train, X_test)

explainer = shap.DeepExplainer(model, torch.tensor(X_train).float())
shap_values = explainer.shap_values(torch.tensor(X_train).float())
shap.force_plot(explainer.expected_value, shap_values, X_test, matplotlib=True)
plt.title("Local Feature Importance with a Force Plot")
plt.show()

\# TODO: \<Enter your answer here\>